## Preparación del entorno


In [ ]:
import pandas as pd
import psycopg2 as ps

# Importa dataset
df = pd.read_csv('Data/DataSouth_Tableau.csv', sep=';')

Mounted at /content/drive


## CRUD completo de un registro

In [ ]:
cur = None
conn = None
table_name = 'airbnb.airbnb'

try:
        # Conectar con una base de datos en elephantsql           
        conn = ps.connect(host="trumpet.db.elephantsql.com", database="critgais",
        user="critgais", password="5VuIHBm7GAt37_36hfnkYUR903_jvWIl")
        
        # Crea un cursor para ejecutar consultas
        cur = conn.cursor()
        
        # Crea un registro 
        print("\n1. Creación de nuevo registro.")
        cur.execute(f"INSERT INTO {table_name} (IdAirbnb, Neighbourhood, City, Latitude, Longitude, Zipcode, PropertyType, RoomType, BedType, Beds, Bedrooms, Bathrooms, Accommodates, Amenities, Price, SecurityDeposit, CleaningFee, CancellationPolicy, ReviewScoresRating, Market, Internet, TV, Kitchen, Airconditioning, Heating) VALUES (9470167, 'Chamartín', 'Madrid', 40.4773, -3.6949, 28015, 'Apartment', 'Private room', 'Real Bed', 1, 1, 1, 2, 'Air conditioning', 30, 150, 10, 'flexible', 95, 'Madrid', TRUE, TRUE, TRUE, TRUE, TRUE)")
        conn.commit()

        # Obtiene el ultimo registro con lastval() y guarda el id en una variable
        cur.execute("SELECT lastval()")
        last_id = cur.fetchone()[0]
        print("\n2. Último registro insertado: ", last_id)

        # Actualización columna camas a 10
        print("\n3. Actualización del número de camas.")
        cur.execute(f"SELECT beds FROM {table_name} WHERE id = {last_id}")
        print("Camas iniciales ", cur.fetchone()[0])
        
        cur.execute(f"UPDATE {table_name} SET beds = 10 WHERE id = {last_id}")
        conn.commit()
        
        cur.execute(f"SELECT beds FROM {table_name} WHERE id = {last_id}")
        print("Camas actualizadas ", cur.fetchone()[0])
        
        # Borrar el registro con el id
        cur.execute(f"DELETE FROM {table_name} WHERE id = {last_id}")
        conn.commit()
        
        # Comprobar que se ha borrado
        cur.execute(f"SELECT EXISTS(SELECT 1 FROM {table_name} WHERE id = {last_id})")
        exists = cur.fetchone()[0]
        if exists:
                print(f'\n4. El registro con id {last_id} todavía existe.')
        else:
                print(f'\n4. El registro con id {last_id} se ha borrado con éxito.')


except Exception as e:
        print(e)

finally:
        # Cerrar la conexión
        if cur is not None:   
                cur.close()
        if conn is not None:
                conn.close()


1. Creación de nuevo registro.

2. Último registro insertado:  8677

3. Actualización del número de camas.
Camas iniciales  1.0
Camas actualizadas  10.0

4. El registro con id 8677 se ha borrado con éxito.


## Consulta datos



In [ ]:
cur = None
conn = None
table_name = 'airbnb.airbnb'

try:
        # Conectar con una base de datos en elephantsql           
        conn = ps.connect(host="trumpet.db.elephantsql.com", database="critgais",
        user="critgais", password="5VuIHBm7GAt37_36hfnkYUR903_jvWIl")
        
        # Crea un cursor para ejecutar consultas
        cur = conn.cursor()
        
        print("\n1. Distritos con tipo de propiedad en Madrid:\n")
        cur.execute(f"SELECT Neighbourhood, PropertyType, COUNT(*) FROM {table_name} WHERE City = 'Madrid' GROUP BY Neighbourhood, PropertyType  ORDER BY COUNT(*)")
        rows = cur.fetchall()
        print(pd.DataFrame(rows, columns=['Neighbourhood', 'PropertyType', 'Count']))

        print("\n2. Cantidad de reviews con más del 95%\n")
        cur.execute(f"SELECT COUNT(*) FROM {table_name}  WHERE ReviewScoresRating > 95")
        rows = cur.fetchall()
        for row in rows:
            print("Total: ", row[0])

        print("\n3. Tipos de política de cancelación y cantidad de pisos por cada uno de ellos. Ordenados de mayor a menor.\n")    
        cur.execute(f"SELECT CancellationPolicy, COUNT(*) FROM {table_name} GROUP BY CancellationPolicy ORDER BY COUNT(*) DESC")
        rows = cur.fetchall()
        print(pd.DataFrame(rows, columns=['CancellationPolicy', 'Número de propiedades'])) 

        print("\n4. Creación de una nueva columna con el cambio de divisa de dólares a euros.\n")    
        cur.execute(f"ALTER TABLE {table_name} ADD COLUMN IF NOT EXISTS PriceEuro FLOAT")
        cur.execute(f"UPDATE {table_name} SET PriceEuro = Price * 0.85")
        conn.commit()
        cur.execute(f'SELECT Price as Dólares, PriceEuro as Euros FROM {table_name} LIMIT 5')
        rows = cur.fetchall()
        for row in rows:
          print("Precio en dólares: ${:,.2f} - Precio en euros: €{:,.2f}".format(*row))
        

except Exception as e:
        print(e)

finally:
        # Cerrar la conexión
        if cur is not None:
                cur.close()
        if conn is not None:
                conn.close()


1. Distritos con tipo de propiedad en Madrid:

            Neighbourhood PropertyType  Count
0               Vicálvaro    Apartment      8
1       Villa de Vallecas    Apartment     24
2                 Barajas    Apartment     36
3              Villaverde    Apartment     46
4               Moratalaz    Apartment     48
5   Fuencarral - El Pardo    Apartment     55
6   San Blas - Canillejas    Apartment     56
7                   Usera    Apartment     79
8               Hortaleza    Apartment     84
9      Puente de Vallecas    Apartment    111
10          Ciudad Lineal    Apartment    160
11            Carabanchel    Apartment    170
12              Chamartín    Apartment    195
13                 Latina    Apartment    214
14      Moncloa - Aravaca    Apartment    224
15                 Tetuán    Apartment    254
16                 Retiro    Apartment    271
17             Arganzuela    Apartment    480
18              Salamanca    Apartment    517
19               Chamberí    Apa

## Conexión y modelado de datos

In [ ]:
# Al ejecutar este script se crea nuevamente la estructura de la base de datos
restart = False

if restart: 
  cur = None
  conn = None

  try:

          # Extrae la columna 'Amenities' e imprimir aquellas filas que superen los 255 caracteres
          df['Amenities'] = df['Amenities'].astype(str).apply(lambda x: x[:255])

          # Cambiar nombre de la columna 'ID' por 'IdAirbnb'
          df.rename(columns={'ID': 'IdAirbnb'}, inplace=True)

          # Conectar con una base de datos en elephantsql           
          conn = ps.connect(host="trumpet.db.elephantsql.com", database="critgais",
          user="critgais", password="5VuIHBm7GAt37_36hfnkYUR903_jvWIl")

          # Crea un cursor para ejecutar consultas
          cur = conn.cursor()

          # Borrar tabla airbnb si existe
          cur.execute("DROP TABLE IF EXISTS airbnb.airbnb")
          conn.commit()

          # Crear schema airbnb si no existe
          cur.execute("CREATE SCHEMA IF NOT EXISTS airbnb")
          conn.commit()

          # Crea una lista con los nombres de las columnas del dataset
          column_names_list = list(df.columns.values)

          # Crea una lista con los tipos de datos de las columnas del dataset
          # y los convierte en los tipos de datos de PostgreSQL
          column_types_list = list(df.dtypes)
          for i in range(len(column_types_list)):
                  if column_types_list[i] == 'int64':
                          column_types_list[i] = 'INT'
                  elif column_types_list[i] == 'float64':
                          column_types_list[i] = 'FLOAT'
                  elif column_types_list[i] == 'bool':
                          column_types_list[i] = 'BOOLEAN'
                  elif column_types_list[i] == 'object':
                          column_types_list[i] = 'VARCHAR(512)'
                  else:
                          column_types_list[i] = 'VARCHAR(512)'

          # Crear una cadena con la definición de la tabla
          table_definition = ', '.join(
                  [f'{col_name} {col_type}' for col_name, col_type in zip(column_names_list, column_types_list)])

          # Añadir a la table airbnb un nuevo campo id como clave primaria que sea autoincremental y que no acepte valores nulos
          table_definition = 'id SERIAL PRIMARY KEY NOT NULL, ' + table_definition

          # Crear la tabla
          cur.execute(f'CREATE TABLE IF NOT EXISTS airbnb.airbnb ({table_definition})')
          conn.commit()

          # Crear string de la inserción con el nombre de las columnas
          columns_query = ', '.join(column_names_list)


          # Inicializar una lista para almacenar las consultas SQL
          values_query_list = []

          # Iterar sobre las filas del dataframe
          for index, row in df.iterrows():
                  # Iterar sobre cada elemento de la fila
                  values_query = []
                  for i in range(len(row)):
                          if type(row[i]) == str:
                                  values_query.append(f"'{row[i]}'")
                          else:
                                  values_query.append(row[i])
                  values_query = ', '.join(map(str, values_query))

                  # Crear una consulta SQL para cada fila
                  insert_query = f"INSERT INTO airbnb.airbnb ({columns_query}) VALUES ({values_query})"
                  cur.execute(insert_query, df.values.tolist())
                  conn.commit()

  except Exception as e:
           print("Se ha producido el siguiente error: ", e)

  finally:
          # Cerrar la conexión
          if cur is not None:
                  cur.close()
          if conn is not None:
                  conn.close()